# Hierarchical Indexing

In [6]:
import numpy as np
import pandas as pd
data = pd.Series(np.random.randn(9),
index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
[1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.688448
   2   -1.446859
   3   -0.396741
b  1   -0.459797
   3   -0.660349
c  1   -1.618859
   2    0.343179
d  2    0.773968
   3    0.499805
dtype: float64

In [5]:
data['b']

1   -1.347772
3   -1.123948
dtype: float64

In [11]:
data[:, 2]

a   -1.446859
c    0.343179
d    0.773968
dtype: float64

In [12]:
data.unstack() #Can be converted into data frame with unstack

,1,2,3
a,0.688448,-1.446859,-0.396741
b,-0.459797,NaN,-0.660349
c,-1.618859,0.343179,NaN
d,NaN,0.773968,0.499805


In [14]:
data.unstack().stack() #Inverse is stack

a  1    0.688448
   2   -1.446859
   3   -0.396741
b  1   -0.459797
   3   -0.660349
c  1   -1.618859
   2    0.343179
d  2    0.773968
   3    0.499805
dtype: float64

- With data frame, both axis can have hierarchical index

In [20]:
frame = pd.DataFrame(np.arange(12).reshape(4,3), index = [['a', 'a', 'b', 'b'], [1,2,1,2]], columns = [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [23]:
frame.index.names = ['Key1', 'Key2']   #Hierarchical index can have names 
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
Key1 Key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [33]:
frame['Ohio','Green']

Key1  Key2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Green), dtype: int32

Reordering and sorting levels

In [36]:
frame.swaplevel('Key1', 'Key2')

state      Ohio     Colorado
color     Green Red    Green
Key2 Key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

# Combining and merging datasets

In [37]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data': range(7)})
df1

,key,data
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [38]:
df2 = pd.DataFrame({'key':['a', 'b', 'd'], 'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [41]:
pd.merge(df1, df2, on ='key')

,key,data,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [44]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [45]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


If column names are different, we can specify them seperately

In [47]:
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey') #default is inner join, so c and d are dropped 

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [48]:
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey', how = 'outer')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


## Many-to-many merges

In [51]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [52]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [53]:
pd.merge(df1, df2, on = 'key', how = 'left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [54]:
pd.merge(df1, df2, how = 'inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [55]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
....: 'key2': ['one', 'two', 'one'],
....: 'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
....: 'key2': ['one', 'one', 'one', 'two'],
....: 'rval': [4, 5, 6, 7]})

In [56]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [57]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [58]:
pd.merge(left, right, on = ['key1', 'key2'], how = 'outer') # Merge on multiple keys

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [59]:
pd.merge(left, right, on = 'key1') # If two columns have same names, they will be assigned suffix after merge

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [61]:
pd.merge(left, right, on = 'key1', suffixes = ('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


# Merging on index

In [62]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [63]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [64]:
right1

,group_val
a,3.5
b,7.0


In [67]:
pd.merge(left1, right1, left_on = 'key', right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [68]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],'key2': [2000, 2001, 2002, 2001, 2002], 'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)), index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],[2001, 2000, 2000, 2000, 2001, 2002]],columns=['event1', 'event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [69]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [70]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index = True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


Using index from both sides of the merge is also possible

In [72]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],index=['a', 'c', 'e'],columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],index=['b', 'c', 'd', 'e'],columns=['Missouri', 'Alabama'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [73]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [74]:
pd.merge(left2, right2, how = 'outer', left_index = True, right_index = True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


join can also be used to combine DataFrame object with same or similar index but non-overlapping columns

In [77]:
left2.join(right2, how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [79]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


# Reshape and pivot

In [82]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
index=pd.Index(['Ohio', 'Colorado'], name='state'),
columns=pd.Index(['one', 'two', 'three'],
name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [85]:
result = data.stack() # stack = DataFrame -> Series
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [87]:
result.unstack() # with heirarchical indexing, unstack can convert data back to DataFrame

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [88]:
x = pd.DataFrame(np.arange(10).reshape(2, 5))
x

,0,1,2,3,4
0,0,1,2,3,4
1,5,6,7,8,9


In [93]:
result.unstack(0) # Unstack innermost level by default, can unstack a different level by passing a name

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [99]:
result.unstack(-1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [103]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [104]:
data2.unstack() # Missing values if not all values in a level are found n each subgroup

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [105]:
data2.unstack().stack() #stacking by default remove NaN values

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [106]:
data2.unstack().stack(dropna=False) # unless dropna = False is sepcified

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [107]:
df = pd.DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [111]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [112]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

# Pivoting long to wide

In [169]:
data = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [170]:
# PeriodIndex combines year and quarter columns to create a time interval type
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [175]:
pivoted = ldata.pivot(index = 'date', columns = 'item', values = 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [176]:
ldata['value2'] = np.random.randn(len(ldata))
ldata.head()

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.385058
1,1959-03-31 23:59:59.999999999,infl,0.000,0.986485
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.055689
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.355129
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.651877


In [188]:
lpivoted = ldata.pivot(index = 'date', columns = 'item') #Ignoring the value argument gives hirarchical columns
lpivoted.head()

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.986485  0.385058   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.651877 -0.355129   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.902640  3.192159   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.010113  0.273606   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.111534  1.078768   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.055689  
1959-06-30 23:59:59.999999999 -0.410071  
1959-09-30 23:59:59.999999999 -0.104999  
1959-12-31 23:59:59.999999999  0.076220  
1960-03-31 23:59:59.999999999  0.117580

In [190]:
unstacked = ldata.set_index(['date', 'item']).unstack('item') # pivot is equal to set_index then unstack(columns)
unstacked.head()

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.986485  0.385058   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.651877 -0.355129   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.902640  3.192159   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.010113  0.273606   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.111534  1.078768   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.055689  
1959-06-30 23:59:59.999999999 -0.410071  
1959-09-30 23:59:59.999999999 -0.104999  
1959-12-31 23:59:59.999999999  0.076220  
1960-03-31 23:59:59.999999999  0.117580

# Pivoting from wide to long

In [192]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
'A': [1, 2, 3],
'B': [4, 5, 6],
'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [200]:
melted = pd.melt(df, id_vars=['key'], var_name='letters', value_name='number')
melted

,key,letters,number
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [203]:
pd.melt(df, id_vars=['key'], value_vars=['A']) # columns not in value_vars are dropped, by default retains all columns

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
